In [1]:
import subprocess
from bs4 import BeautifulSoup
import requests
import os
import time
from tqdm import tqdm

In [2]:
contest_id = '1800'
problem_id = 'A'
directory = contest_id + '/' + problem_id
replace = {'&quot;': '\"', '&gt;': '>', '&lt;': '<', '&amp;': '&', "&apos;": "'"}
languages = {'GNU C++20 (64)', 'GNU C++14', 'GNU C++11', 'GNU C++17'} # , 'Java 8':'Java 11.0.5', 'Python 3': 'Python 3.7.2', 'Python 2':'Python 2.7.15'

if (not os.path.exists(directory)):
    os.makedirs(directory)
if (not os.path.exists(os.path.join(directory,'compiled'))):
    os.makedirs(os.path.join(directory,'compiled'))

In [3]:
def parse(code):
	for key in replace.keys():
		code = code.replace(key, replace[key])
	return code

#fetches the code corresponding to the parameter url
def getCode(submission_url, submission_id, contest_id):
    source = requests.get(submission_url).text
    soup = BeautifulSoup(source, 'html.parser')
    language = soup.findAll('td')[3].text[6:-6]
    if language not in languages:
        return
    code = soup.findAll('pre')[0].text
    code = parse(code).replace('\r', '')
    if code != '':
        print("Sucessfully fetched", submission_id + ".cpp")
        filewrite = open(directory + '/' + submission_id + '.cpp', 'w')
        filewrite.write(code)
        filewrite.close()

In [ ]:
page = 5
max_page = 287
url = "http://codeforces.com/contest/"+contest_id+"/status/"+problem_id+"/page/" + str(page) + "?order=BY_ARRIVED_DESC"
print(url)
for i in tqdm(range(page,max_page+1)):
    url = "http://codeforces.com/contest/"+contest_id+"/status/"+problem_id+"/page/" + str(page) + "?order=BY_ARRIVED_DESC"
    print ("Parsing page:", page)
    try:
        source = requests.get(url, timeout=3.5).text
        soup = BeautifulSoup(source, 'html.parser')
        submissions = soup.findAll('a', {'class':'view-source'})
        
        for submission in submissions:
            submission_url = "http://codeforces.com" + submission['href']
            try:
                getCode(submission_url, submission['submissionid'], contest_id)
            except:
                print("Failed to fetch:", submission_url)
                continue
            time.sleep(1)

    except:
        print("Failed parsing on page:", page)

    page+=1

In [4]:
for file in tqdm(os.listdir(directory)):
    file_name = os.path.splitext(file)[0]
    file_ext = os.path.splitext(file)[1]
    try:
        os.system('g++ ' + os.path.join(directory, file_name)+file_ext + ' -o ' + os.path.join(directory,'compiled',file_name) + ' -O2')    
    except:
        continue

100%|██████████| 1037/1037 [13:52<00:00,  1.25it/s]


In [1]:
with open('input.txt', 'r') as myfile: 
    test_input = myfile.read()
with open('output.txt', 'r') as myfile: 
    test_output = myfile.read()

pref = contest_id +'\\'+problem_id
hackable = []
rte = []
for file in tqdm(os.listdir(pref + '\compiled')):
    print(file)
    cmd = 'more input.txt | ' + pref + '\compiled\\' + file
    try:
        out = subprocess.check_output(cmd, shell=True, text=True)
        out = out.lower()
        # print(out)
        out = out.rstrip()
        if (out == test_output):
            # print(out)
            hackable.append(os.path.splitext(file)[0])
    except:
        rte.append(file)
        
for r in rte:
    print("Ran into runtime exception:", file)

print("Hackable:" )
for h in hackable:
    print('https://codeforces.com/contest/' +contest_id + '/submission/' +  h)

NameError: name 'contest_id' is not defined